In [61]:
import numpy as np
import os
import pandas as pd

## Data making    

In [62]:
def data_rate(s_c,e_c,s_p,e_p):
    k1 = 0.65*2
    k2 = 0.35*2
    rate = k1*abs((s_c - s_p)/(e_p-s_p)) + k2*abs((e_p - e_c)/(e_p-s_p))
    return rate
## waste rate  越大越不好
Car_n = 1000
Park_n = 1200

In [144]:
count = '5_1200'
path = 'Data/'+ count

In [145]:
Car_time = pd.read_csv(path+'Car_time.csv')

Park_time = pd.read_csv(path+'Park_time.csv')

User_cost = pd.read_csv(path+'User_cost.csv')

User_PreList = pd.read_csv(path+'User_PreferenceList.csv')

In [146]:
Pre_ID = []
for i in range(Car_n):
    temp = []
    for j in range(Park_n):
        k = int(User_PreList.loc[i][j].split(',')[0][1:])
        temp.append(k)
    Pre_ID.append(temp)

In [147]:
Car_Time = np.array(Car_time)
Park_Time = np.array(Park_time)

In [148]:
ranklist = Pre_ID

In [149]:
Car_Time
Park_Time
len(ranklist[1])


1200

### Slide Window

In [150]:
Suc_match = []
Matching = []
Car_sta = []
Park_sta = []
for i in range(Park_n):
    if i < Car_n:
        Car_sta.append(0)
    Park_sta.append(0)
    #initiate the list

In [151]:
def find_index(car_index):
    index = -1
    for i in range(len(Matching)):
        if Matching[i][1] == car_index:
            index = i
            break
    return index 

In [152]:
def find_car_index(park_index):
    car_index = -1
    for i in Matching:
        if i[2] == park_index:
            car_index = i[1]
            break
    return car_index 

In [153]:
def Modify(car_index):##DC-G-S
    #print(car_index)
    #print(Car_sta[car_index])
    now_park_index = ranklist[car_index][Car_sta[car_index]] - 1
    if Park_sta[now_park_index] == 0:
        #print('find_index:',find_index(car_index))
        Matching[find_index(car_index+1)] = [Car_Time[car_index][0],car_index+1,now_park_index+1]
        Park_sta[now_park_index] = 1
        return 
    if find_car_index(now_park_index+1) == -1:
        Car_sta[car_index] = Car_sta[car_index] + 1
        Modify(car_index)
    else:
        index = find_car_index(now_park_index+1) - 1
        now = data_rate(Car_Time[car_index][0],Car_Time[car_index][1],Park_Time[now_park_index][0],Park_Time[now_park_index][1])
        ago = data_rate(Car_Time[index][0],Car_Time[index][1],Park_Time[now_park_index][0],Park_Time[now_park_index][1])
        if now < ago:
            Car_sta[index] = Car_sta[index] + 1
            Modify(index)
        else:
            Car_sta[car_index] = Car_sta[car_index] + 1
            Modify(car_index)

In [154]:
def Modify1(car_index):## FIFO
    #print(car_index)
    #print(Car_sta[car_index])
    now_park_index = ranklist[car_index][Car_sta[car_index]] - 1
    if Park_sta[now_park_index] == 0:
        #print('find_index:',find_index(car_index))
        Matching[find_index(car_index+1)] = [Car_Time[car_index][0],car_index+1,now_park_index+1]
        Park_sta[now_park_index] = 1
        return 
    Car_sta[car_index] = Car_sta[car_index] + 1
    Modify1(car_index)
    '''if find_car_index(now_park_index+1) == -1:
        Car_sta[car_index] = Car_sta[car_index] + 1
        Modify(car_index)
    else:
        index = find_car_index(now_park_index+1) - 1
        now = data_rate(Car_Time[car_index][0],Car_Time[car_index][1],Park_Time[now_park_index][0],Park_Time[now_park_index][1])
        ago = data_rate(Car_Time[index][0],Car_Time[index][1],Park_Time[now_park_index][0],Park_Time[now_park_index][1])
        if now < ago:
            Car_sta[index] = Car_sta[index] + 1
            Modify(index)
        else:
            Car_sta[car_index] = Car_sta[car_index] + 1
            Modify(car_index)'''

In [155]:
def Match(car):
    Matching.append(car)
    Car_index = car[1]-1
    Modify(Car_index)

In [156]:
def Match1(car):
    Matching.append(car)
    Car_index = car[1]-1
    Modify1(Car_index)

In [157]:
point =0.012

In [158]:
Matching = []
Suc_match = []
for i in range(Park_n):
    if i < Car_n:
        Car_sta.append(0)
    Park_sta.append(0)
time1 = 0.0
time2 = point
for i in range(Car_n*2):
    if i < Car_n:
        tempdata = [Car_Time[i][0],i+1,ranklist[i][0]]
        Match(tempdata)
    
    count = 0
    for j in range(len(Matching)):
        if Matching[j][0] >= time1 and Matching[j][0] <= time2:
            Suc_match.append(Matching[count])
            count = count+1
    Matching = Matching[count:]
    if len(Suc_match) != 0:
        for k in Suc_match:
            if Car_Time[k[1]-1][1] >= time1 and Car_Time[k[1]-1][1] <= time2:
                Park_sta[k[2]-1] = 0
                #Park_Time[k[2]-1][0] = Car_Time[k[1]-1][1]
    time1 = time1 + point
    time2 = time2 + point
    '''input()
    print(time1,'            ',time2)
    print(i,':Matching:',Matching)
    print('Suc_match:',Suc_match)'''

In [159]:
len(Matching)

0

In [160]:
len(Suc_match)

1000

In [161]:
def Index(car,park):
    ind = 0
    for i in range(len(ranklist[car])):
        if park == ranklist[car][i]:
            ind = i + 1
            break
    return ind

In [162]:
Match_num = []
Unique = []
for i in Suc_match:
    Match_num.append(Index(i[1]-1,i[2]))
    Unique.append(i[2])

In [163]:
num = len(np.unique(Unique))
num

933

In [164]:
sss = sorted(Match_num)

In [165]:
import matplotlib.pyplot as plt

In [166]:
'''plt.plot(sorted(Match_num))
plt.xlabel('Car ID')
plt.ylabel('Preference Number')
plt.savefig('/home/sunny/Paper/image/9.jpg')
'''

"plt.plot(sorted(Match_num))\nplt.xlabel('Car ID')\nplt.ylabel('Preference Number')\nplt.savefig('/home/sunny/Paper/image/9.jpg')\n"

In [167]:
#Suc_match

In [168]:
Sum_rate = 0
for i in range(len(Suc_match)):
    rate = abs(Car_Time[Suc_match[i][1]-1][1] - Car_Time[Suc_match[i][1]-1][0])/abs(Park_Time[Suc_match[i][2]-1][1] - Park_Time[Suc_match[i][2]-1][0])
    #print(rate)
    Sum_rate = Sum_rate + rate
print(Sum_rate/num)

0.3595035451361873


In [169]:
print(num)

933


In [170]:
print(sum(sss)/len(sss))

1.857
